<a href="https://colab.research.google.com/github/arif238291/DecisionTree-RandomForest-Model/blob/main/code_2_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%reset -f


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip -q install pyomo pandas highspy
# backup solver (GLPK) in case HiGHS isn’t available for any reason
!apt-get -qq update
!apt-get -qq install -y glpk-utils

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
# Nodes(charging Station candidate sites) data file created
nodes = pd.DataFrame({
    "id": list(range(1, 16)),
    "name": [
        "Beijing North Railway Station",
        "Peking University People's Hospital",
        "Beijing Jiaotong University",
        "Beijing Zoo",
        "Beijing Univ. of Civil Eng. & Architecture",
        "Capital Stadium",
        "Fuchengmen",
        "Xinjiekou",
        "Jishuitan",
        "Navy Hospital",
        "Capital Normal University",
        "South of Baishiqiao",
        "North Campus of Beijing Normal University",
        "Beihai Hospital",
        "Beihai Park"
    ],
    "Cj": [   # Table 2 basic construction cost
        52000, 57000, 54600, 49000, 52500,
        49000, 45400, 47000, 44000, 56000,
        55000, 49000, 53000, 53000, 49500
    ],
    "PD": [1.0]*15,                # Table 3: all PD = 1.0
    "lambda_mu_factor": [1.0]*15   # keep =1.0 unless heterogeneous arrivals
})


# Path in Google Drive
file_path = "/content/drive/My Drive/colab_note/location_optimization//data/nodes.csv"

# Save to CSV
nodes.to_csv(file_path, index=False)


In [ ]:
# Demand data
demand = pd.DataFrame({
    "i": [1, 2, 3, 4],
    "name": ["Town 1", "Town 2", "Town 3", "Town 4"],
    "mi": [1600, 1500, 2000, 1300],  # total BEVs per town
    "alpha": [1.0]*4,
    "beta": [1.0]*4,
    "QkWh": [0.25]*4                 # Table 3: 0.25 kWh/session
})

# Distances with numeric station IDs
distances = pd.DataFrame([
    # Town 1
    {"i": 1, "j": 1,  "dij_km": 11.11},
    {"i": 1, "j": 2,  "dij_km": 9.52},
    {"i": 1, "j": 3,  "dij_km": 9.90},
    {"i": 1, "j": 4,  "dij_km": 8.85},
    {"i": 1, "j": 5,  "dij_km": 8.26},
    {"i": 1, "j": 6,  "dij_km": 9.43},
    {"i": 1, "j": 7,  "dij_km": 9.17},
    {"i": 1, "j": 8,  "dij_km": 9.90},
    {"i": 1, "j": 9,  "dij_km": 10.87},
    {"i": 1, "j": 10, "dij_km": 8.20},
    {"i": 1, "j": 11, "dij_km": 9.09},
    {"i": 1, "j": 12, "dij_km": 9.90},
    {"i": 1, "j": 13, "dij_km": 8.93},
    {"i": 1, "j": 14, "dij_km": 9.80},
    {"i": 1, "j": 15, "dij_km": 9.01},

    # Town 2
    {"i": 2, "j": 1,  "dij_km": 10.53},
    {"i": 2, "j": 2,  "dij_km": 10.53},
    {"i": 2, "j": 3,  "dij_km": 11.24},
    {"i": 2, "j": 4,  "dij_km": 9.09},
    {"i": 2, "j": 5,  "dij_km": 10.10},
    {"i": 2, "j": 6,  "dij_km": 8.26},
    {"i": 2, "j": 7,  "dij_km": 8.77},
    {"i": 2, "j": 8,  "dij_km": 7.25},
    {"i": 2, "j": 9,  "dij_km": 8.70},
    {"i": 2, "j": 10, "dij_km": 9.80},
    {"i": 2, "j": 11, "dij_km": 8.70},
    {"i": 2, "j": 12, "dij_km": 8.20},
    {"i": 2, "j": 13, "dij_km": 7.58},
    {"i": 2, "j": 14, "dij_km": 8.62},
    {"i": 2, "j": 15, "dij_km": 9.80},

    # Town 3
    {"i": 3, "j": 1,  "dij_km": 6.76},
    {"i": 3, "j": 2,  "dij_km": 7.30},
    {"i": 3, "j": 3,  "dij_km": 7.25},
    {"i": 3, "j": 4,  "dij_km": 7.46},
    {"i": 3, "j": 5,  "dij_km": 6.21},
    {"i": 3, "j": 6,  "dij_km": 7.94},
    {"i": 3, "j": 7,  "dij_km": 8.13},
    {"i": 3, "j": 8,  "dij_km": 7.41},
    {"i": 3, "j": 9,  "dij_km": 7.58},
    {"i": 3, "j": 10, "dij_km": 7.52},
    {"i": 3, "j": 11, "dij_km": 8.20},
    {"i": 3, "j": 12, "dij_km": 7.09},
    {"i": 3, "j": 13, "dij_km": 8.20},
    {"i": 3, "j": 14, "dij_km": 7.81},
    {"i": 3, "j": 15, "dij_km": 7.81},

    # Town 4
    {"i": 4, "j": 1,  "dij_km": 12.35},
    {"i": 4, "j": 2,  "dij_km": 12.82},
    {"i": 4, "j": 3,  "dij_km": 10.87},
    {"i": 4, "j": 4,  "dij_km": 11.11},
    {"i": 4, "j": 5,  "dij_km": 11.11},
    {"i": 4, "j": 6,  "dij_km": 10.99},
    {"i": 4, "j": 7,  "dij_km": 10.87},
    {"i": 4, "j": 8,  "dij_km": 14.71},
    {"i": 4, "j": 9,  "dij_km": 10.20},
    {"i": 4, "j": 10, "dij_km": 11.11},
    {"i": 4, "j": 11, "dij_km": 10.99},
    {"i": 4, "j": 12, "dij_km": 11.24},
    {"i": 4, "j": 13, "dij_km": 11.24},
    {"i": 4, "j": 14, "dij_km": 12.05},
    {"i": 4, "j": 15, "dij_km": 9.80},
])







#distances = pd.DataFrame(dist_rows)

# Save to Drive
demand_path = "/content/drive/My Drive/colab_note/location_optimization/data/demand.csv"
demand.to_csv(demand_path, index=False)

distances_path = "/content/drive/My Drive/colab_note/location_optimization/data/distances.csv"
distances.to_csv(distances_path, index=False)


In [ ]:
%%writefile ev_station_siting.py


import os  #import os
import math # access mathematical operations beyond python
import pandas as pd

from pyomo.environ import (
    ConcreteModel, Var, Objective, Constraint, Set, Param, NonNegativeIntegers,
    Binary, minimize, value, SolverFactory, Reals
) # ConcreteModel~Pyomo optimization model

# ----------------------------
# 1) Configuration
# ----------------------------
CONFIG = {
    # Planning knobs
    "P": 7,                 # number of stations to open
    "TD": 99.0,              # tolerable distance (km)
    "C_access": 0.40,       # $ per km access cost to reach destination place
    "Pc": 12000.0,          # $ per charger
    "Nmin": 2,              # minimum chargers if a site is opened
    # Discounting/annualization is assumed embedded in Cj and Pc for simplicity

    # Optional M/M/s pre-sizing -> lower bound on chargers N_j
    "use_queue_presize": True,
    "target_wait_hours": 0.5,  # wait cap (E[Wq] + service might be simplified)
    # Simple queue inputs (scale arrivals by node factor):
    "lambda_base_per_hr": 6.0,  # base arrivals/hour if site serves "typical" load
    "mu_per_hr": 2.0,           # service rate per charger (sessions/hour/charger)


    # Input files
    "nodes_csv": "/content/drive/My Drive/colab_note/location_optimization/data/nodes.csv",
    "demand_csv": "/content/drive/My Drive/colab_note/location_optimization/data/demand.csv",
    "dist_csv": "/content/drive/My Drive/colab_note/location_optimization/data/distances.csv",

    # Solver preference list
    "solvers": ["appsi_highs", "highs", "glpk"],

    # New: hard upper bound for chargers to keep model linear and force N[j]=0 when X[j]=0
    "Nmax": 50
}

# cfg is a local name CONFIG
def load_or_make_data(cfg):
    nodes = pd.read_csv(cfg["nodes_csv"])
    demand = pd.read_csv(cfg["demand_csv"])
    distances = pd.read_csv(cfg["dist_csv"])
    return nodes, demand, distances

# ---------------------------------------------------
# 2) Simple M/M/s helper (Erlang C) for modelling queueing at charger station
#  (pre-sizing s (no. of chargers))
# ---------------------------------------------------
def erlang_c_wait(lambda_rate, mu_rate, s):
    """
    Returns (P_wait, E[Wq]) for M/M/s with arrival rate lambda_rate (λ),
    per-server service rate mu_rate (μ), and s chargers.
    E[Wq] = (P_wait) / (s*μ - λ)
    """
    if s <= 0 or mu_rate <= 0 or lambda_rate <= 0:
        return 0.0, 0.0
    rho = lambda_rate / (s * mu_rate)
    if rho >= 1.0:
        return 1.0, float('inf')

    # Compute Erlang C (probability that arriving customer must wait)
    # P0 (normalization)
    sum_terms = sum((lambda_rate/mu_rate)**n / math.factorial(n) for n in range(s))
    last_term = ((lambda_rate/mu_rate)**s) / (math.factorial(s) * (1 - rho))
    P0 = 1.0 / (sum_terms + last_term)
    P_wait = last_term * P0
    Wq = P_wait / (s * mu_rate - lambda_rate)  # in hours if rates are per hour
    return P_wait, Wq

def minimal_servers_for_wait(lambda_rate, mu_rate, wait_cap_hours, s_max=100):
    """
    Find smallest s such that E[Wq] <= wait_cap_hours.
    """
    if lambda_rate <= 0:
        return 0
    for s in range(1, s_max+1):
        _, Wq = erlang_c_wait(lambda_rate, mu_rate, s)
        if Wq <= wait_cap_hours:
            return s
    return s_max

# ----------------------------
# 3) Build the Pyomo model
# ----------------------------
def build_model(nodes_df, demand_df, dist_df, cfg):
    m = ConcreteModel()

    # Index sets
    m.J = Set(initialize=list(nodes_df["id"]))      # candidate station sites
    m.I = Set(initialize=list(demand_df["i"]))      # demand zones

    # Parameters (dictionaries to Params)
    id_to_Cj = dict(zip(nodes_df["id"], nodes_df["Cj"]))
    id_to_PD = dict(zip(nodes_df["id"], nodes_df["PD"]))
    id_to_fac = dict(zip(nodes_df["id"], nodes_df["lambda_mu_factor"]))

    i_to_mi   = dict(zip(demand_df["i"], demand_df["mi"]))
    i_to_a    = dict(zip(demand_df["i"], demand_df["alpha"]))
    i_to_b    = dict(zip(demand_df["i"], demand_df["beta"]))
    i_to_Q    = dict(zip(demand_df["i"], demand_df["QkWh"]))

    # Distance matrix
    # Create allowed pairs filtered by TD
    dij = {}
    allowed_pairs = []
    for _, row in dist_df.iterrows():
        i, j, d = int(row["i"]), int(row["j"]), float(row["dij_km"])
        dij[(i, j)] = d
        if d <= cfg["TD"]:
            allowed_pairs.append((i, j))

    m.dij = Param(m.I, m.J, initialize=lambda mdl, i, j: dij[(i, j)], within=Reals)

    # Scalars
    m.P = Param(initialize=cfg["P"])
    m.C_access = Param(initialize=cfg["C_access"])
    m.Pc = Param(initialize=cfg["Pc"])
    m.Nmin = Param(initialize=cfg["Nmin"])
    m.Nmax = Param(initialize=cfg["Nmax"])

    # Per-node / per-demand Params
    m.Cj = Param(m.J, initialize=id_to_Cj)
    m.PD = Param(m.J, initialize=id_to_PD)
    m.factor = Param(m.J, initialize=id_to_fac)

    m.mi = Param(m.I, initialize=i_to_mi)
    m.alpha = Param(m.I, initialize=i_to_a)
    m.beta = Param(m.I, initialize=i_to_b)
    m.Q = Param(m.I, initialize=i_to_Q)

    # Decision variables
    m.X = Var(m.J, within=Binary)                      # build station at j?
    m.Y = Var(m.I, m.J, within=Binary)                 # assign i -> j ?
    m.N = Var(m.J, within=NonNegativeIntegers)         # number of chargers at j

    # Optional queue-based *lower bound* on N_j
    # We compute a per-node baseline s_min_j using lambda_base * factor[j]
    presize = {}
    if cfg["use_queue_presize"]:
        for j in m.J:
            lam = cfg["lambda_base_per_hr"] * float(m.factor[j])
            smin = minimal_servers_for_wait(
                lambda_rate=lam,
                mu_rate=cfg["mu_per_hr"],
                wait_cap_hours=cfg["target_wait_hours"]
            )
            presize[j] = smin
    else:
        for j in m.J:
            presize[j] = 0

    # Precompute allowed assignment pairs (distance <= TD)
    allowed = set(allowed_pairs)

    #  Objective components:
    #  Station cost: sum_j Cj * X_j + Pc * N_j      (linearized)
    #  User cost:    sum_{i,j in allowed} mi * [ beta_i * C_access * d_ij + alpha_i * PD_j * Q_i ] * Y_ij

    def obj_rule(mdl):
        station_cost = sum(mdl.Cj[j] * mdl.X[j] + mdl.Pc * mdl.N[j] for j in mdl.J)
        user_cost = 0
        for i in mdl.I:
            for j in mdl.J:
                if (i, j) in allowed:
                    access_term = mdl.beta[i] * mdl.C_access * mdl.dij[i, j]
                    energy_term = mdl.alpha[i] * mdl.PD[j] * mdl.Q[i]
                    per_user = access_term + energy_term
                    user_cost += mdl.mi[i] * per_user * mdl.Y[i, j]
        return station_cost + user_cost

    m.OBJ = Objective(rule=obj_rule, sense=minimize)

    # Constraints

    # (1) Each demand zone assigned to exactly one station
    def one_station_rule(mdl, i):
        return sum(mdl.Y[i, j] for j in mdl.J if (i, j) in allowed) == 1
    m.AssignOnce = Constraint(m.I, rule=one_station_rule)

    # (2) Exactly P stations opened
    def p_stations_rule(mdl):
        return sum(mdl.X[j] for j in mdl.J) == mdl.P
    m.PStations = Constraint(rule=p_stations_rule)

    # (3) Linking: can only assign to opened stations
    def link_rule(mdl, i, j):
        if (i, j) in allowed:
            return mdl.Y[i, j] <= mdl.X[j]
        else:
            # If outside TD, force Y=0 by fixing upper bound 0
            return mdl.Y[i, j] <= 0
    m.Link = Constraint(m.I, m.J, rule=link_rule)

    # (4) Minimum chargers if opened; also queue-based lower bound
    def chargers_min_rule(mdl, j):
        return mdl.N[j] >= max(mdl.Nmin, presize[j]) * mdl.X[j]
    m.MinChargersIfOpen = Constraint(m.J, rule=chargers_min_rule)

    # (5) Hard upper bound so N[j]==0 when X[j]==0 (keeps model linear)
    def chargers_max_rule(mdl, j):
        return mdl.N[j] <= mdl.Nmax * mdl.X[j]
    m.MaxChargersIfClosed = Constraint(m.J, rule=chargers_max_rule)

    # (Optional) You can add a soft upper bound like N[j] <= Nmax * X[j] if desired
    # (Optional) Capacity constraints (time-based) can be added later if you want
    return m, presize

# ----------------------------
# 4) Solve and pretty-print
# ----------------------------
def solve_and_report(model, cfg, nodes_df, demand_df, dist_df, presize):
    # Choose a solver that exists
    solver = None
    for sname in cfg["solvers"]:
        try:
            solver = SolverFactory(sname)
            if solver.available():
                print(f"Using solver: {sname}")
                break
        except Exception:
            pass
    if solver is None or not solver.available():
        raise RuntimeError("No suitable MILP solver found. Install HiGHS (highspy) or GLPK.")

    res = solver.solve(model, tee=False)

    # Extract solution
    X_sol = {j: round(value(model.X[j])) for j in model.J}
    opened = [j for j, x in X_sol.items() if x == 1]
    Y_sol = {(i, j): round(value(model.Y[i, j])) for i in model.I for j in model.J}
    N_sol = {j: int(round(value(model.N[j]))) for j in model.J}
    total_cost = value(model.OBJ)

    # Display
    print("\n=== Optimal Solution ===")
    print(f"Total cost: ${total_cost:,.2f}")
    print(f"Stations opened (|J|={len(model.J)}, P={model.P.value}): {opened}")
    print("\nPer-site details:")
    for j in model.J:
        nodename = nodes_df.loc[nodes_df["id"] == j, "name"].values[0]
        print(f"  Site j={j:>2} ({nodename:>8}): X={X_sol[j]}  N_chargers={N_sol[j]}  (pre-size floor={presize[j]})")

    print("\nAssignments (i -> j):")
    for i in model.I:
        for j in model.J:
            if Y_sol[(i, j)] == 1:
                iname = demand_df.loc[demand_df["i"] == i, "name"].values[0]
                jname = nodes_df.loc[nodes_df["id"] == j, "name"].values[0]
                dij = dist_df[(dist_df["i"] == i) & (dist_df["j"] == j)]["dij_km"].values[0]
                print(f"  {iname} (i={i}) -> {jname} (j={j})   d={dij:.2f} km")

# ----------------------------
# 5) Main
# ----------------------------
if __name__ == "__main__":
    nodes, demand, distances = load_or_make_data(CONFIG)
    model, presize = build_model(nodes, demand, distances, CONFIG)
    solve_and_report(model, CONFIG, nodes, demand, distances, presize)


Overwriting ev_station_siting.py


In [ ]:
from pathlib import Path

p = Path("ev_station_siting.py")
code = p.read_text()

# 1) Add Reals to the imports from pyomo.environ
code = code.replace(
    "Binary, minimize, value, SolverFactory",
    "Binary, minimize, value, SolverFactory, Reals"
)

# 2) Replace the incorrect 'within=float' with the proper Pyomo domain
code = code.replace("within=float", "within=Reals")

p.write_text(code)
print("Patched ev_station_siting.py")


Patched ev_station_siting.py


In [ ]:
!python ev_station_siting.py

Using solver: appsi_highs

=== Optimal Solution ===
Total cost: $691,282.00
Stations opened (|J|=15, P=7): [4, 6, 7, 8, 9, 12, 15]

Per-site details:
  Site j= 1 (Beijing North Railway Station): X=0  N_chargers=0  (pre-size floor=4)
  Site j= 2 (Peking University People's Hospital): X=0  N_chargers=0  (pre-size floor=4)
  Site j= 3 (Beijing Jiaotong University): X=0  N_chargers=0  (pre-size floor=4)
  Site j= 4 (Beijing Zoo): X=1  N_chargers=4  (pre-size floor=4)
  Site j= 5 (Beijing Univ. of Civil Eng. & Architecture): X=0  N_chargers=0  (pre-size floor=4)
  Site j= 6 (Capital Stadium): X=1  N_chargers=4  (pre-size floor=4)
  Site j= 7 (Fuchengmen): X=1  N_chargers=4  (pre-size floor=4)
  Site j= 8 (Xinjiekou): X=1  N_chargers=4  (pre-size floor=4)
  Site j= 9 (Jishuitan): X=1  N_chargers=4  (pre-size floor=4)
  Site j=10 (Navy Hospital): X=0  N_chargers=0  (pre-size floor=4)
  Site j=11 (Capital Normal University): X=0  N_chargers=0  (pre-size floor=4)
  Site j=12 (South of Baishiqia